In [ ]:
url = "https://pypi.org/project/pandas/#history"

from gazpacho import get, Soup
import pandas as pd



html = get(url)
soup = Soup(html)
cards = soup.find("a", {"class": "card"})
cards[0].find("p", attrs={"class": "release__version"}, partial=False)

def parse_card(card):
    version = card.find("p", {"class": "release__version"}, partial=False).text
    timestamp = card.find("time").attrs['datetime']
    return {"version": version, 'timestamp': timestamp}

df = pd.DataFrame([parse_card(c) for c in cards]).assign(timestamp=lambda d: pd.to_datetime(d['timestamp']))
df

In [ ]:
from gazpacho import get, Soup

url = 'https://scrape.world/books'
html = get(url)
soup = Soup(html)
books = soup.find('div', {'class': 'book-'}, partial=True)

def parse(book):
    name = book.find('h4').text
    price = float(book.find('p').text[1:].split(' ')[0])
    return {name: price}

[parse(book) for book in books]

In [ ]:
from gazpacho import get, Soup
import pandas as pd
import numpy as np


url_maccabi = "https://www.maccabi.co.il/season.asp"

html = get(url_maccabi)
soup = Soup(html)
games = soup.find("div", {"class": "line he"})


def parser(game):
    def find_date(heb_date: str) -> str:
        convert_mounth = {"ינואר": '01', "פברואר": '02', "מרץ": '03', "אפריל": '04', "מאי": '05', "יוני": '06', "יולי": '07', "אוגוסט": '08', "ספטמבר": '09', "אוקטובר": '10', "נובמבר": '11', "דצמבר": '12'}
        split_date = heb_date.split()
        zero = '0' if len(split_date[0]) == 1 else ''
        return zero + split_date[0] + '/' + convert_mounth[split_date[1][1:]] + '/2023'

    game_data = game.find("div", {"class": "gameDate"}).strip().split()
    translate_ = {'בית': 'home', 'חוץ': 'out', 'ווינר-סל': 'winner', 'יורוליג': 'euroligue'}
    date_game = find_date(game.find("div", {"class": "gameDate"}).find("strong").text)
    day_game = game_data[2] if date_game != None else np.nan
    cycle = game_data[-1]
    league = translate_[game.find("div", {"class": "leagueIcon"}).html.split('alt=')[1].split('>')[0].replace('"', '').split()[-1]] if game.find("div", {"class": "leagueIcon"}).html.split('alt=')[1].split('>')[0].replace('"', '').split()[-1] in translate_ else np.nan
    return {'date game': date_game, 'day game': day_game, 'cycle': cycle, 'league': league}#, 'hospitality': hospitality}#, 'league': league}

    
df = pd.DataFrame([parser(game) for game in games])
df


In [56]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

url_maccabi = "https://www.maccabi.co.il/season.asp"
html = requests.get(url_maccabi).content
soup = BeautifulSoup(html, "html.parser", from_encoding="iso-8859-8")
games = soup.find_all("div", {"class": "line he"})
games

def parser(game: BeautifulSoup):
    game_data = game.find("div", {"class": "gameDate"}).text.split()
    convert_mounts = {"ינואר": '01', "פברואר": '02', "מרץ": '03', "אפריל": '04', "מאי": '05', "יוני": '06', "יולי": '07', "אוגוסט": '08', "ספטמבר": '09', "אוקטובר": '10', "נובמבר": '11', "דצמבר": '12'}
    complementary_zero = '0' if len(game_data[0]) == 1 else ''
    date_game = complementary_zero + game_data[0] + '/' + convert_mounts[game_data[1][1:]] + '/2023'
    day_game = game_data[2]
    time_game = game_data[3] if len(game_data) == 6 else np.nan
    cycle = game_data[-1]
    league = game.find("div", {"class": "leagueIcon"}).img.attrs['alt']
    home = game.find("div", {"class": "transportation"})
    home = home.img.attrs['alt'] if home.img else np.nan
    return {
        'day game': day_game,
        'cycle': cycle,
        'league': league,
        'date game': date_game,
        'time game': time_game,
        'hospitality': home,
    }

games_parsed = [parser(game) for game in games]
games_parsed_df = pd.DataFrame(games_parsed)
games_parsed_df

Overwriting df_macabbi.py


In [59]:
!pyinstrument df_macabbi.py

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 1012, in _validate_conn
    conn.connect()
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 411, in connect
    self.sock = ssl_wrap_socket(
  File "/usr/lib/python3/dist-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "/usr/lib/python3/dist-packages/urllib3/util/ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "/usr/lib/python3.10/ssl.py", line 513, in wrap_socket
    return self.sslsocket_class._create(
  File "/usr/lib/python3.10/ssl.py", line 1071, in _create
    self.do_hand

In [57]:
games_parsed_df

,date game,day game,cycle,league,time game,hospitality
0,23/02/2023,חמישי,25,יורוליג,21:05,משחק בית
1,02/03/2023,חמישי,26,יורוליג,20:00,משחק חוץ
2,05/03/2023,ראשון,18,ליגת ווינר-סל,18:00,משחק בית
3,08/03/2023,רביעי,27,יורוליג,19:30,משחק בית
4,10/03/2023,שישי,28,יורוליג,20:30,משחק חוץ
...,...,...,...,...,...,...
62,24/09/2023,שבת,הכנה,משחקי הכנה,18:00,משחק חוץ
63,23/09/2023,שישי,הכנה,משחקי הכנה,18:00,משחק חוץ
64,20/09/2023,שלישי,הכנה,משחקי הכנה,17:20,משחק חוץ
65,11/09/2023,ראשון,הכנה,משחקי הכנה,17:15,משחק חוץ


In [45]:
home.img.attrs['alt']

'משחק בית'